In [2]:
# Python Imports
import ipywidgets as widgets
import networkx as nx
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# Project Imports
from src.mesh_analysis.readers.file_reader import FileReader
from src.mesh_analysis.tracers.waku_tracer import WakuTracer

In [14]:
tracer = WakuTracer()
reader = FileReader("../../../test/log_test/", tracer)

df = reader.read()
test = df.reset_index()

In [6]:
%matplotlib
dropdown1 = widgets.SelectMultiple(
                        options=df.index.get_level_values(0).drop_duplicates(),
                        description='Timestamp',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
dropdown2 = widgets.SelectMultiple(
                        options=df.index.get_level_values(1).drop_duplicates(),
                        description='Msg hash',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
    
fig = plt.figure()
ax_ = fig.add_subplot(1, 1, 1)
def test(x, y, data_):
    ax_.cla()
    
    first_timestamp = data_.index.get_level_values(0)[0]
    df_ = data_.loc[(first_timestamp,y[0]):(x[0], y[0])]
    df_.reset_index(inplace=True)
    G = nx.from_pandas_edgelist(df_, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash'], create_using=nx.DiGraph)
    
    timestamp_edges = nx.get_edge_attributes(G, 'timestamp')
    hash_edges = nx.get_edge_attributes(G, 'msg_hash')
    ds = [timestamp_edges, hash_edges]
    d = {}
    for k in timestamp_edges.keys():
        d[k] = tuple(d[k] for d in ds)
    pos = nx.spring_layout(G)
    nx.draw(G, pos, ax=ax_, with_labels=True)
    nx.draw_networkx_edge_labels(G, pos, ax=ax_, edge_labels=d)
    plt.show()


i_plot = widgets.interactive(test, x=dropdown1, y=dropdown2, data_=widgets.fixed(df))
i_plot

Using matplotlib backend: QtAgg


interactive(children=(SelectMultiple(description='Timestamp', layout=Layout(height='100px', width='40%'), opti…

In [70]:
print(df)
data_ = df[df.index.get_level_values(1) == '0x1']

                                       sender_peer_id receiver_peer_id
timestamp                     msg_hash                                
2024-04-22 14:06:58.001+00:00 0x1                   A                B
2024-04-22 14:06:58.002+00:00 0x1                   B                D
2024-04-22 14:06:58.003+00:00 0x1                   A                C
                              0x1                   D                E
2024-04-22 14:06:58.006+00:00 0x1                   A                F
2024-04-22 14:06:59.001+00:00 0x1                   A                B


In [23]:
fig = plt.figure()
ax_ = fig.add_subplot(1, 1, 1)
# 1. 
data_ = df[df.index.get_level_values(1) == '0x1']
data_prepared = data_.reset_index()
# 2. 
complete_graph = nx.from_pandas_edgelist(data_prepared, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash'], create_using=nx.DiGraph)
g_pos_layout = nx.kamada_kawai_layout(complete_graph)
e_pos_layout = nx.draw_networkx_edge_labels(complete_graph, g_pos_layout, ax=ax_)
plt.cla()

# 3. 
first_timestamp = data_.index.get_level_values(0)[0]
df_ = data_.loc[(first_timestamp,'0x1'):('2024-04-22 14:06:58.003+00:00', '0x1')]
df_.reset_index(inplace=True)
selected_graph = nx.from_pandas_edgelist(df_, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash'], create_using=nx.DiGraph)

# 4. 
g_pos_layout_selected = {key: g_pos_layout[key] for key in list(selected_graph.nodes)}
e_pos_layout_selected = {key: e_pos_layout[key] for key in selected_graph.edges}

nx.draw(selected_graph, g_pos_layout_selected, ax=ax_, with_labels=True)
nx.draw_networkx_edge_labels(selected_graph, g_pos_layout_selected, ax=ax_)
plt.show()


In [ ]:
# g_pos_layout, dict:
# 'A' : [0.68431856 0.16921637], etc
# e_pos_layout, dict
# ('A', 'B'): [Pos]



